In [1]:
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser

In [2]:
from git import Repo


In [11]:
! poetry add sentence_transformers

Using version ^2.2.2 for sentence-transformers

Updating dependencies
Resolving dependencies... (6.1s)s://files.pythonhosted.org/packages/09/7f/dd8cae1ed66d43bdff9e0d250b806d2217a9c18177f4f65cc3e4bc907308/sentencepiece-0.1.99-cp310-cp310-macosx_10_9_universal2.whl  55% (3.4s) (2.7s)Resolving dependencies... (3.8s)

Package operations: 13 installs, 0 updates, 0 removals

  • Installing networkx (3.1): Pending...
  • Installing networkx (3.1): Downloading... 0%
  • Installing networkx (3.1): Downloading... 40%
  • Installing networkx (3.1): Downloading... 100%
  • Installing networkx (3.1): Installing...
  • Installing networkx (3.1)
  • Installing joblib (1.3.2): Pending...
  • Installing pillow (10.1.0): Pending...
  • Installing safetensors (0.4.0): Pending...
  • Installing scipy (1.9.3): Pending...
  • Installing threadpoolctl (3.2.0): Pending...
  • Installing torch (2.1.0): Pending...
  • Installing threadpoolctl (3.2.0): Pending...
  • Installing torch (2.1.0): Pending...
  • Ins

In [3]:
!pwd

/Users/jghiya/Documents/unoplat/lamp/llm-rag


In [3]:
repo_path = "/Users/jghiya/Documents/unoplat/test_repo"
#repo = Repo.clone_from("https://github.com/langchain-ai/langchain", to_path=repo_path)

In [4]:
# Load
loader = GenericLoader.from_filesystem(
    repo_path+"/libs/langchain/langchain",
    glob="**/*",
    suffixes=[".py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=500)
)
documents = loader.load()
len(documents)

1871

In [5]:
type(documents)


list

In [42]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
python_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON, 
                                                               chunk_size=1000, 
                                                               chunk_overlap=300)
texts = python_splitter.split_documents(documents)
len(texts)

10149

In [43]:
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings

In [46]:
GPT4AllEmbeddings??

Init signature: GPT4AllEmbeddings(*, client: Any = None) -> None
Source:        
class GPT4AllEmbeddings(BaseModel, Embeddings):
    """GPT4All embedding models.

    To use, you should have the gpt4all python package installed

    Example:
        .. code-block:: python

            from langchain.embeddings import GPT4AllEmbeddings

            embeddings = GPT4AllEmbeddings()
    """

    client: Any  #: :meta private:

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that GPT4All library is installed."""

        try:
            from gpt4all import Embed4All

            values["client"] = Embed4All()
        except ImportError:
            raise ImportError(
                "Could not import gpt4all library. "
                "Please install the gpt4all library to "
                "use this embedding model: pip install gpt4all"
            )
        return values

    def embed_documents(self, texts: List[str]) -> List[List[floa

In [44]:
from langchain.embeddings import HuggingFaceEmbeddings


modelPath = "BAAI/bge-large-en-v1.5"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'mps'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': True}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

In [45]:
#embeddings = OllamaEmbeddings(model="mistral",base_url="http://localhost:11434")
db = Chroma.from_documents(texts, embedding=embeddings)
retriever = db.as_retriever(
    search_type="similarity",  # Also test "similarity"
    search_kwargs={"k": 8},
)

In [46]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler  
from langchain.llms import Ollama

In [47]:
llm = Ollama(model="codellama:34b", 
             callback_manager = CallbackManager([StreamingStdOutCallbackHandler()]))
memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [39]:
question = "What classes are derived from the Chain class?"
retrieved_docs = retriever.invoke("What classes are derived from the Chain class?")
print(len(retrieved_docs))

for doc in retrieved_docs:
    print(doc.page_content)


8
class Chain(RunnableSerializable[Dict[str, Any], Dict[str, Any]], ABC):
    """Abstract base class for creating structured sequences of calls to components.

    Chains should be used to encode a sequence of calls to components like
    models, document retrievers, other chains, etc., and provide a simple interface
    to this sequence.
@property
    def _chain_type(self) -> str:
        return "vector_db_qa_with_sources_chain"
class Chain(RunnableSerializable[Dict[str, Any], Dict[str, Any]], ABC):
    """Abstract base class for creating structured sequences of calls to components.

    Chains should be used to encode a sequence of calls to components like
    models, document retrievers, other chains, etc., and provide a simple interface
    to this sequence.

    The Chain interface makes it easy to create apps that are:
        - Stateful: add Memory to any Chain to give it state,
        - Observable: pass Callbacks to a Chain to execute additional functionality,
            like

In [34]:
question = "How can I initialize a ReAct agent through code?"
result = qa(question)
result['answer']


How can a ReAct agent be initialized through code?
A ReAct agent can be initialized through the `initialize_agent` function provided by LangChain. This function takes in several arguments such as tools, LLM, agent type, callback manager, agent path, agent kwargs, tags, and additional keyword arguments. The agent type argument is set to "react" for a ReAct agent.

To use this function, you can create an instance of the `AgentExecutor` class and pass it the necessary arguments. Here's an example:
```python
from langchain.agents import initialize_agent

def main():
    tools = [Tool1(), Tool2()]
    llm = LLM()
    agent = AgentType.REACT
    callback_manager = None
    agent_path = None
    agent_kwargs = {}
    tags = []

    # Initialize the ReAct agent and start it
    executor = initialize_agent(tools, llm, agent, callback_manager, agent_path, agent_kwargs, tags)
    while True:
        # Call the agent to get output
        output = executor.get_output()
        if output is None:


'\nA ReAct agent can be initialized through the `initialize_agent` function provided by LangChain. This function takes in several arguments such as tools, LLM, agent type, callback manager, agent path, agent kwargs, tags, and additional keyword arguments. The agent type argument is set to "react" for a ReAct agent.\n\nTo use this function, you can create an instance of the `AgentExecutor` class and pass it the necessary arguments. Here\'s an example:\n```python\nfrom langchain.agents import initialize_agent\n\ndef main():\n    tools = [Tool1(), Tool2()]\n    llm = LLM()\n    agent = AgentType.REACT\n    callback_manager = None\n    agent_path = None\n    agent_kwargs = {}\n    tags = []\n\n    # Initialize the ReAct agent and start it\n    executor = initialize_agent(tools, llm, agent, callback_manager, agent_path, agent_kwargs, tags)\n    while True:\n        # Call the agent to get output\n        output = executor.get_output()\n        if output is None:\n            break\n        

In [48]:
questions = [
    #"What is the class hierarchy in general?",
    "What classes are derived from the Chain class in the langchain project?"
    #"What one improvement do you propose in code in relation to the class hierarchy for the Chain class?",
]

for question in questions:
    result = qa(question)
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

 "What are the derived classes from the Chain class in the LangChain project?" The derived classes from the Chain class in the LangChain project are:

1. LLMChain
2. MapReduceDocumentsChain
3. StuffDocumentsChain
4. ReduceDocumentsChain
5. BaseCombineDocumentsChain
6. APIRequesterChain
7. APIResponderChain
8. StringEvaluator

Note that this list may not be exhaustive, as the LangChain project is constantly evolving. The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. The human then asks about the classes derived from the Chain class in the LangChain project, and the AI provides a list of the derived classes.-> **Question**: What classes are derived from the Chain class in the langchain project? 

**Answer**:  The derived classes from the Chain class in the LangChain project are:

1. LLMChain
2. MapReduceDocumentsChain
3. StuffDocumentsChain
4. ReduceDocumentsChain

In [42]:
retrieved_docs = retriever.invoke("Which child classes inherit from the Chain class?")
print(len(retrieved_docs))


8
class ReduceDocumentsChain(BaseCombineDocumentsChain):
    """Combine documents by recursively reducing them.

    This involves

    - combine_documents_chain

    - collapse_documents_chain

    `combine_documents_chain` is ALWAYS provided. This is final chain that is called.
    We pass all previous results to this chain, and the output of this chain is
    returned as a final result.

    `collapse_documents_chain` is used if the documents passed in are too many to all
    be passed to `combine_documents_chain` in one go. In this case,
    `collapse_documents_chain` is called recursively on as big of groups of documents
    as are allowed.

    Example:
        .. code-block:: python

            from langchain.chains import (
                StuffDocumentsChain, LLMChain, ReduceDocumentsChain
            )
            from langchain.prompts import PromptTemplate
            from langchain.llms import OpenAI


In [43]:

for doc in retrieved_docs:
    print("Starting doc")
    print(doc.page_content)
    print("Ending doc")

Starting doc
class ReduceDocumentsChain(BaseCombineDocumentsChain):
    """Combine documents by recursively reducing them.

    This involves

    - combine_documents_chain

    - collapse_documents_chain

    `combine_documents_chain` is ALWAYS provided. This is final chain that is called.
    We pass all previous results to this chain, and the output of this chain is
    returned as a final result.

    `collapse_documents_chain` is used if the documents passed in are too many to all
    be passed to `combine_documents_chain` in one go. In this case,
    `collapse_documents_chain` is called recursively on as big of groups of documents
    as are allowed.

    Example:
        .. code-block:: python

            from langchain.chains import (
                StuffDocumentsChain, LLMChain, ReduceDocumentsChain
            )
            from langchain.prompts import PromptTemplate
            from langchain.llms import OpenAI
Ending doc
Starting doc
class MultiRetrievalQAChain(MultiRout